In [43]:
#Creating the functions
import pandas as pd

#load in the dataframes for functions
lemmaDF = pd.read_csv("Dictionary_Dataframes/lemmas.csv", sep = "{")
del lemmaDF['Unnamed: 0']
parsesDF = pd.read_csv("Dictionary_Dataframes/parses.csv", sep = "{")
del parsesDF["Unnamed: 0"]
#outputs list of matching IDs or an empty list
def get_id(word):
    #search the lemmaDF for the word
    output = []
    #this is the dataframe of all rows with the word
    df = lemmaDF.loc[lemmaDF["bare_text"] == word]
        #iterrows works, but the others dont. ok whatever
    for index, row in df.iterrows():
        output.append(row["id"])
    return(output)
def get_id_parse(word, input):
    #search parseDF for instances and add to output if it isn't already in there
    output = input
    df = parsesDF.loc[parsesDF["bare_text"] == word]
    for index, row in df.iterrows():
        id = row["id"]
        if id not in output:
            output.append(id)
    return output

#outouts dictionary of ids to the valid words 
def get_words_help(ids):
    output = {}
    if(type(ids) == list):
        for id in ids:
            curOut = []
            df = parsesDF.loc[parsesDF["id"] == id]
            for index, row in df.iterrows():
                #can add a checker for POS or add info about POS
                curOut.append(row["bare_text"])
            output[id] = curOut
        return output
       #for each id in the list, grab all the words with matching ID from parses and put in "outputs"
       #outputs is dict of ID --> parses with ID 
    else:
        print("Something went wrong...")
        return {}
    #enter FPP
def get_id_full(word):
    ids = get_id(word)
    ids = get_id_parse(word, ids)
    return ids
def get_words(word):
    ids = get_id_full(word)
    return get_words_help(ids)
def check_enclitic(word, pos):
    if(pos != "verb"):
        if(word.endswith('st')):
            return True
    if((word.endswith("ve"))
    or (word.endswith("ne"))
    or (word.endswith("que"))
    or (word.endswith("ue")) 
    or word.endswith("ce")):
        return True
    else:
        return False
def get_def(id):
    output = ""
    df = lemmaDF.loc[lemmaDF["id"] == id]
    for index, row in df.iterrows():
        output = (row["definition"])
    return output


#function from word -> dictionary entry
def get_dict(word, show_def = False):
    ids = get_id_full(word)
        #list of dictionary entries 
    output = {}
    for id in ids:
        pos = "" 
        dictEntry = ""
        rowText = ""
        morph = ""
        text = {}
        definition = get_def(id)
        df = parsesDF.loc[parsesDF["id"] == id]
        for index, row in df.iterrows():
            rowText = row["bare_text"]
            morph = row["morph_code"]
            #noun case
                #check required to prevent substantive adj from outputting as nouns 
            if (row["morph_code"][0] == "n" and (pos == "" or pos == "noun")):
                pos = "noun"
                text[2] = row["morph_code"][6] 
                #make the nom, singular --> text[0]
                if((row["morph_code"][2] == "s") and (row["morph_code"][7] == "n")):
                    #have to hard-code around que enclitic 
                    if(not check_enclitic(row["bare_text"], pos)):
                        text[0] = row["bare_text"]
                #make the gen, singular --> text[1]
                if((row["morph_code"][2] == "s") and (row["morph_code"][7] == "g")):
                    if(not check_enclitic(row["bare_text"], pos)):
                        text[1] = row["bare_text"]
            
            #adj case 
            if (row["morph_code"][0] == "a"):
                pos = "adj"
                #for each, need the singular, positive, nominative - without enclitics
                if((not check_enclitic(row["bare_text"], pos)) and type(row["misc_features"]) == type(1.0) ):
                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n") 
                    and (row["morph_code"][6] == "m")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        text[0] = row["bare_text"]

                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n") 
                    and (row["morph_code"][6] == "f")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        text[1] = row["bare_text"]

                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n") 
                    and (row["morph_code"][6] == "n")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        text[2] = row["bare_text"]
            #verb case - need t morph code for the 4PP
            if((row["morph_code"] == "t-sr-pmn-") and not (check_enclitic(row["bare_text"], "verb"))):text[3] = row["bare_text"] 
            if((row["morph_code"][0] == "v")): 
                pos = "verb"
                kind = "regular"
                morph = row["morph_code"]
                curWord = row["bare_text"]
                #make 3 cases: regular, deponent, and inpersonal - may be more ask Celano
                #0: present first person singular active indicative: v1spia---
                #1: present active infinitive: v--pna---
                #2: perfect first person singular active indicative: v1sria---
                #3: t-sr-pmn- is the morph code --> has 3PP if deponent 
                #4: present third person singular active indicative - for impersonal verbs
                #5: grab stuff with deponent voice tag v1spip---
                if((not check_enclitic(curWord, pos)) and type(row["misc_features"]) == type(1.0)):
                    if(morph) == "v1spia---":
                        text[0] = curWord
                    if(morph) == "v--pna---":
                        text[1] = curWord
                    if(morph) == "v1sria---":
                        text[2] = curWord
                    if(morph) == "v3spia---":
                        text[4] = curWord
                    if(morph) == "v1spip---":
                        text[5] = curWord
                    
            #all other cases
        #preventing errors <3 
        if(type(definition) != type("str") or len(definition) == 0):
            definition = "No Definiton"
        for i in range(10):
            if(not i in text):
                text[i] = "NULL"
        #make dictionary entry string
        match(pos):
            #make sure each of those exists before trying to output
            case "noun":
                if(show_def):
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + ". - " + (definition)
                else:
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] 
            case "adj":
                if(show_def):
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + " - " + definition
                else:
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] 
            case "verb":
                #change which type by checking if we have a normal FPP
                if(text[0]!= "NULL"):
                    kind = "regular"
                elif(text[5] != "NULL"):
                    kind = "deponent"
                else:
                    kind = "impersonal"
                    #nested switch block! Yay!
                match(kind):
                    case "regular":
                        if(show_def):
                            dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[0] + ", " + text[1] + ", "+ text[2] + ", " + text[3]
                    case "deponent":
                        if(show_def):
                            dictEntry = pos + ": " + text[5] + ", " + text[1] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[5] + ", " + text[1] + ", " + text[3]
                    case "impersonal":
                        if(show_def):
                            dictEntry = pos + ": " + text[4] + ", " + text[1] +", " + text[2] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[4] + ", " + text[1] +", " + text[2]+  ", "+ text[3]
            case other:
                dictEntry = "Unexpected Error: unknown POS"
        output[id] = (dictEntry)
    return output

Run block 1 to initialize functions in kernel

get_words(str word) - returns a list of all words with that input in its list of parses

get_dict(str word, bool print_def = False) - returns all dictionary entries for a given word. 
    Gives definition if print_def is true



In [45]:
#check if the code runs in another block
# print(get_id("mater"))
# print(get_words_help([46163]))
#print(get_words("populus"))
print(get_dict("insigni", True))

{44781: 'noun: NULL, insignis, n. - a mark, indication, proof, sign, token, signal', 44782: 'verb: insignio, insignire, NULL, insignitus- to mark, make conspicuous, distinguish', 44783: 'adj: insignis, insignis, NULL - distinguished by a mark, remarkable, noted, eminent, distinguished, prominent, extraordinary'}
